In [86]:
# DIANNA AGUSTINA S.
# NIM. 1202912003
# S1 KK SISTEM INFORMASI

# UAS BIG DATA

#1 RFM

In [99]:
import pandas as pd
import matplotlib.pyplot as plb
import squarify
import numpy as np
from datetime import timedelta

data = pd.read_csv('../input/uasrfmdatadianna/UAS_RFM_Dianna.csv')
data.head()

,CardID,TrxDate,Amount
0,1000152,15/03/21,561174
1,1000158,17/03/21,515021
2,1000156,18/02/21,481722
3,1000115,08/04/21,453926
4,1000164,18/01/21,127328


In [100]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   CardID   199 non-null    int64 
 1   TrxDate  199 non-null    object
 2   Amount   199 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ KB


In [101]:
data['TrxDate'] = pd.to_datetime(data['TrxDate'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 KB


In [102]:
data.shape

(199, 3)

In [103]:
data.isnull().sum()

CardID     0
TrxDate    0
Amount     0
dtype: int64

In [104]:
snapshot_date = data['TrxDate'].max() + timedelta(days=1)

data_process = data.groupby(['CardID']).agg({
    'TrxDate': lambda x: (snapshot_date - x.max()).days,
    'CardID': 'count',
    'Amount': 'sum'
})

data_process.rename(columns={
    'TrxDate': 'Recency',
    'CardID': 'Frequency',
    'Amount': 'Monetary'
}, inplace=True)

print(data_process.head())
print('{:,} rows; {:,} columns'
     .format(data_process.shape[0], data_process.shape[1]))

         Recency  Frequency  Monetary
CardID                               
1000101       92          1    896744
1000102      201          3    869831
1000103      315          1    362392
1000104      186          2    929049
1000105      174          5   3325243
71 rows; 3 columns


In [105]:
q_recency_1 = np.quantile(data_process['Recency'], 1)
q_recency_2 = np.quantile(data_process['Recency'], 0.75)
q_recency_3 = np.quantile(data_process['Recency'], 0.5)
q_recency_4 = np.quantile(data_process['Recency'], 0.25)

print('Quantile Recency 1 : ', q_recency_1, ' hari')
print('Quantile Recency 2 : ', q_recency_2, ' hari')
print('Quantile Recency 3 : ', q_recency_3, ' hari')
print('Quantile Recency 4 : ', q_recency_4, ' hari')

Quantile Recency 1 :  315  hari
Quantile Recency 2 :  202.0  hari
Quantile Recency 3 :  168.0  hari
Quantile Recency 4 :  94.5  hari


In [106]:
q_freq_1 = np.quantile(data_process['Frequency'], 0.25)
q_freq_2 = np.quantile(data_process['Frequency'], 0.5)
q_freq_3 = np.quantile(data_process['Frequency'], 0.75)
q_freq_4 = np.quantile(data_process['Frequency'], 1)

print('Quantile Frequency 1 : ', q_freq_1, ' kali')
print('Quantile Frequency 2 : ', q_freq_2, ' kali')
print('Quantile Frequency 3 : ', q_freq_3, ' kali')
print('Quantile Frequency 4 : ', q_freq_4, ' kali')

Quantile Frequency 1 :  2.0  kali
Quantile Frequency 2 :  3.0  kali
Quantile Frequency 3 :  3.0  kali
Quantile Frequency 4 :  7  kali


In [107]:
q_monet_1 = np.quantile(data_process['Monetary'], 0.25)
q_monet_2 = np.quantile(data_process['Monetary'], 0.5)
q_monet_3 = np.quantile(data_process['Monetary'], 0.75)
q_monet_4 = np.quantile(data_process['Monetary'], 1)

print('Quantile Monetary 1 : Rp ', q_monet_1.round(1))
print('Quantile Monetary 2 : Rp ', q_monet_2.round(1))
print('Quantile Monetary 3 : Rp ', q_monet_3.round(1))
print('Quantile Monetary 4 : Rp ', q_monet_4.round(1))

Quantile Monetary 1 : Rp  854567.5
Quantile Monetary 2 : Rp  1342488.0
Quantile Monetary 3 : Rp  1732767.0
Quantile Monetary 4 : Rp  4640126


In [108]:
r_labels = range(4,0,-1)
r_groups = pd.qcut(data_process['Recency'], q=4, labels=r_labels)

f_labels = range(1,4)
f_groups = pd.qcut(data_process['Frequency'], q=4, labels=f_labels, duplicates='drop')

m_labels = range(1,5)
m_groups = pd.qcut(data_process['Monetary'], q=4, labels=m_labels)

data_process = data_process.assign(R = r_groups.values, F = f_groups.values, M = m_groups.values)

data_process

,Recency,Frequency,Monetary,R,F,M
CardID,,,,,,
1000101,92,1,896744,4,1,2
1000102,201,3,869831,2,2,2
1000103,315,1,362392,1,1,1
1000104,186,2,929049,2,1,2
1000105,174,5,3325243,2,3,4
...,...,...,...,...,...,...
1000169,292,1,525846,1,1,1
1000170,207,2,957487,1,1,2
1000171,165,1,920539,3,1,2


In [109]:
data_process.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71 entries, 1000101 to 1000174
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Recency    71 non-null     int64   
 1   Frequency  71 non-null     int64   
 2   Monetary   71 non-null     int64   
 3   R          71 non-null     category
 4   F          71 non-null     category
 5   M          71 non-null     category
dtypes: category(3), int64(3)
memory usage: 2.8 KB


In [110]:
data_process['RFM Value'] = data_process['R'].astype(str) + " " + data_process['F'].astype(str) + " " + data_process['M'].astype(str)

data_process

,Recency,Frequency,Monetary,R,F,M,RFM Value
CardID,,,,,,,
1000101,92,1,896744,4,1,2,4 1 2
1000102,201,3,869831,2,2,2,2 2 2
1000103,315,1,362392,1,1,1,1 1 1
1000104,186,2,929049,2,1,2,2 1 2
1000105,174,5,3325243,2,3,4,2 3 4
...,...,...,...,...,...,...,...
1000169,292,1,525846,1,1,1,1 1 1
1000170,207,2,957487,1,1,2,1 1 2
1000171,165,1,920539,3,1,2,3 1 2


In [112]:
data_rfm = data_process.groupby(['RFM Value']).agg({
    'RFM Value': 'count'
})

data_rfm.rename(columns={
    'RFM Value': 'Count'
}, inplace=True)

data_rfm

,Count
RFM Value,
1 1 1,8
1 1 2,3
1 1 3,2
1 2 2,1
1 2 3,1
1 2 4,3
2 1 1,3
2 1 2,4
2 1 3,2
